In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3076/3076), done.
remote: Total 4084 (delta 1185), reused 1985 (delta 948), pack-reused 0
Receiving objects: 100% (4084/4084), 44.62 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.


In [ ]:
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cd models/research && \
    cp object_detection/packages/tf2/setup.py . && \
    python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import io
import imageio
import os
import pathlib
from PIL import Image
import time

In [ ]:
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(
    fname='mscoco_label_map.pbtxt',
    origin=url,
    untar=False)


5056/5056 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step


In [ ]:
!head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


In [ ]:
label_idx=label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS
)

In [ ]:
label_idx

In [ ]:
import tensorflow_hub as hub

In [ ]:
model = hub.load("https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1")

In [ ]:
def load_image(Image_path):
  return np.array(Image.open(Image_path))

In [ ]:
def Image_Detection(image_np):
  input_tensor=tf.constant(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  detections=model(input_tensor)
  num_detections=int(detections.pop('num_detections'))
  detections={key:value[0,:num_detections].numpy() for key,value in detections.items()}
  detections['num_detections']=num_detections
  detections['detection_classes']=detections['detection_classes'].astype(np.int64)
  image_with_detections=image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      label_idx,
      use_normalized_coordinates=True,
      min_score_thresh=0.3,
      max_boxes_to_draw=200
      )
  return image_with_detections

In [ ]:
import cv2
import numpy as np

In [ ]:
def detect_objects_in_video(input_video):
  video_reader=imageio.get_reader(f'{input_video}.mp4')
  video_writer=imageio.get_writer(f'{input_video}_annotated.mp4')
  t0=time.time()
  n_frames=0
  for frame in video_reader:
    n_frames+=1
    new_frame=Image_Detection(frame)
    video_writer.append_data(new_frame)
  fps=n_frames/(time.time()-t0)
  print("Frames processed: %s, Speed: %s fps" % (n_frames, fps))
  video_writer.close()

In [ ]:
detect_objects_in_video("safari")

In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/cnn_class2_videos.zip


--2024-03-24 10:47:25--  https://lazyprogrammer.me/course_files/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  --.-KB/s    in 0.08s   

2024-03-24 10:47:26 (23.8 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]



In [ ]:
!unzip cnn_class2_videos.zip

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             
